In [1]:
import numpy as np
import pandas as pd

QUANTILE_CONST = -1.644854
PORTFOLIO_VOLUME = 1000

SCALE = 439735 / 78525044

In [2]:
df = pd.read_csv('all.csv', parse_dates=['date'])
df.sort_values(by='date', ascending=True, inplace=True)

train = df[:200].reset_index(drop=True)
test = df[200:].reset_index(drop=True)

In [3]:
ret = train['mid'][1:].reset_index(drop=True) - train['mid'][:-1].reset_index(drop=True)

ret_mean, ret_std = np.mean(ret), np.std(ret)

# 95% VaR
var = -(QUANTILE_CONST * ret_std + ret_mean)

In [4]:
lix_train = np.log10(train['volume'] * train['mid'] * SCALE / (train['max_ask'] - train['min_bid']))

# Predict worst LIX with 95% confidence level from historical data
lix_mean, lix_std = np.mean(lix_train), np.std(lix_train)

lix = QUANTILE_CONST * lix_std + lix_mean

# Average price
p_mid = train['mid'].mean()

# Calculate cost of liquidity
col = PORTFOLIO_VOLUME * p_mid * (10 ** -lix) / 2

In [5]:
lvar = var + col
col

15.182898859683553

In [6]:
ret_test = test['mid'][1:].reset_index(drop=True) - test['mid'][:-1].reset_index(drop=True)

def count_exceptions(returns, var):
    count = np.count_nonzero(returns < -var)
    rate = count / len(returns)
    return count, rate

count_exceptions(ret_test, lvar)

(12, 0.14634146341463414)